In [84]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from sklearn import ensemble
from scipy.sparse import coo_matrix
from sklearn import svm

import xgboost
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

In [85]:
import keras.backend as K

In [86]:
def w2v_average_wiki():
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))
    songs=utils.w2v_average_wiki(songs,'glove-wiki-gigaword-200')
    return np.array(songs),np.array(labels)

In [87]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [88]:
X,y=w2v_average_wiki()

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000002C941C96908>>
Traceback (most recent call last):
  File "C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000002C95D85E2E8>>
Traceback (most recent call last):
  File "C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._se

In [89]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5)

In [90]:
def dummy(X_train,X_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(X_train, y_train)
    y_pred=dummy.predict(X_test)
    return f1_score(y_test,y_pred)

In [91]:
print(dummy(X_train,X_test,y_train,y_test))

0.43546730571722014


In [92]:
def logistics_sentiment():
    conn=sqlite3.connect('SONGS.db')
    data=pd.read_sql("select Economic,neg,pos,neu,comp from song",conn).dropna()
    X=data[['neg','pos','neu','comp']]
    y=data.Economic
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred=logreg.predict(X_test)
    return f1_score(y_test,y_pred)

In [93]:
print(logistics_sentiment())

0.0


C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [94]:
def logistics(X_train,X_test,y_train,y_test):
    logreg = LogisticRegression()
    logreg.fit(X_train,y_train)
    y_pred=logreg.predict(X_test)
    return f1_score(y_test,y_pred)

In [95]:
print(logistics(X_train,X_test,y_train,y_test))

0.44390832328106156

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [96]:
def naive_bayes(X_train,X_test,y_train,y_test):
    nb=GaussianNB()
    nb.fit(X_train, y_train)
    y_pred=nb.predict(X_test)
    return f1_score(y_test,y_pred)

In [97]:
print(naive_bayes(X_train,X_test,y_train,y_test))

0.5605167087896658


In [98]:
def support_vector(X_train,X_test,y_train,y_test):
    model = svm.SVC()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [99]:
print(support_vector(X_train,X_test,y_train,y_test))

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.0


C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [100]:
def knn(X_train,X_test,y_train,y_test):
    model = KNeighborsClassifier(n_neighbors=5) 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [101]:
print(knn(X_train,X_test,y_train,y_test))

0.5021048603138154


In [102]:
def decision_tree(X_train,X_test,y_train,y_test):
    model = DecisionTreeClassifier(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [103]:
print(decision_tree(X_train,X_test,y_train,y_test))

0.6039004707464695


In [104]:
def random_forest(X_train,X_test,y_train,y_test):
    model=ensemble.RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [105]:
print(random_forest(X_train,X_test,y_train,y_test))

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5603414823438105


In [106]:
def xgb(X_train,X_test,y_train,y_test):
    X_train=coo_matrix(X_train).tocsc()
    X_test=coo_matrix(X_test).tocsc()
    model=xgboost.XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [107]:
print(xgb(X_train,X_test,y_train,y_test))

0.5361867704280155


In [108]:
def simple_nn(X_train,X_test,y_train,y_test):

    def create_model_architecture(input_size):
        input_layer = layers.Input(shape=(input_size,),)
        hidden_layer = layers.Dense(200, activation="relu")(input_layer)
        output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)
        classifier = models.Model(inputs = input_layer, outputs = output_layer)
        classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
        return classifier

    model=create_model_architecture(len(X_train[0]))
    model.fit(X_train, y_train,epochs=100)
    y_pred = [int(round(i[0])) for i in model.predict(X_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [109]:
print(simple_nn(X_train,X_test,y_train,y_test))

Epoch 1/100
3327/3327 [==============================] - 1s 158us/step - loss: 0.6845
Epoch 2/100
3327/3327 [==============================] - 0s 76us/step - loss: 0.6698
Epoch 3/100
3327/3327 [==============================] - 0s 78us/step - loss: 0.6601
Epoch 4/100
3327/3327 [==============================] - 0s 73us/step - loss: 0.6580
Epoch 5/100
3327/3327 [==============================] - 0s 77us/step - loss: 0.6464
Epoch 6/100
3327/3327 [==============================] - 0s 76us/step - loss: 0.6342
Epoch 7/100
3327/3327 [==============================] - 0s 74us/step - loss: 0.6258
Epoch 8/100
3327/3327 [==============================] - 0s 68us/step - loss: 0.6157
Epoch 9/100
3327/3327 [==============================] - 0s 68us/step - loss: 0.6048
Epoch 10/100
3327/3327 [==============================] - 0s 68us/step - loss: 0.5931
Epoch 11/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.5831
Epoch 12/100
3327/3327 [==============================] - 0s 6

3327/3327 [==============================] - 0s 72us/step - loss: 0.1102
Epoch 97/100
3327/3327 [==============================] - 0s 71us/step - loss: 0.1019
Epoch 98/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.1016
Epoch 99/100
3327/3327 [==============================] - 0s 72us/step - loss: 0.1016
Epoch 100/100
3327/3327 [==============================] - 0s 70us/step - loss: 0.0970
0.6096328730212193
